# Second assing of LLM group from 2024.2
The goal of this project is to compare the diference of speed between the use of sequential calls and async
*Made by Puca Vaz*


## Defining groq calls 

In [7]:
from groq import AsyncGroq

client_groq = AsyncGroq()

In [8]:
async def groq_call(client, prompt, model):
    completion = await client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=False,
        stop=None,
    )

    return completion.choices[0].message.content


## Using prompts 

In [9]:
prompts = [ "Escreva uma frase sobre a importância da leitura.", 
           "Resuma os principais temas abordados no livro Capitães de areia do autor Jorge Amado e explique como eles se relacionam com a sociedade atual", 
           "Elabore um ensaio que analise as implicações éticas e sociais do uso de inteligência artificial na tomada de decisões judiciais. Discuta os potenciais benefícios e riscos, citando exemplos reais ou hipotéticos, e proponha diretrizes que poderiam ajudar a equilibrar eficiência tecnológica com justiça humana"]

models = ["llama-3.2-1b-preview", "llama-3.1-8b-instant", "llama-3.2-90b-text-preview" ]

In [10]:
import asyncio

In [11]:
responses = {}

async def async_groq_calls(prompts, models, client_groq):
    for prompt in prompts:
        responses[prompt] = {}
        print("Prompt:", prompt)
        tasks = []
        for model in models:
            print("Scheduling call for model:", model)
            tasks.append(groq_call(client_groq, prompt, model))
        
        results = await asyncio.gather(*tasks)
        
        
        for model, response in zip(models, results):
            print(f"Response from model {model}:")
            print(response)
            responses[prompt][model] = response
            print("\n\n")


In [12]:
await async_groq_calls(prompts, models, client_groq)


Prompt: Escreva uma frase sobre a importância da leitura.
Scheduling call for model: llama-3.2-1b-preview
Scheduling call for model: llama-3.1-8b-instant
Scheduling call for model: llama-3.2-90b-text-preview
Response from model llama-3.2-1b-preview:
A leitura desempenha um papel fundamental na educação e na aceleração do desenvolvimento intelectual, proporcionando acesso a uma vasta gama de conhecimentos, culturas e experiências, que podem abrir portas para novas perspectivas e uma vida mais rica e satisfatória.



Response from model llama-3.1-8b-instant:
A leitura é uma ferramenta fundamental para o desenvolvimento da mente, pois ela estimula a imaginação, aumenta a fluência linguística, aprimora a compreensão e a memória, e proporciona uma visão ampla do mundo, capacitando as pessoas a tomar decisões informadas e a se tornar mais cidadãos ativos e conscientes.



Response from model llama-3.2-90b-text-preview:
A leitura é uma ferramenta essencial para o desenvolvimento pessoal e int

## Avaliator

In [22]:
from openai import AsyncOpenAI   
client_openai = AsyncOpenAI()

In [23]:
async def avaliar_resposta(client, pergunta_usuario, resposta_usuario):
    criterios = """
    Avalie a resposta fornecida com base nos seguintes critérios:

    1. **Relevância ao Prompt:** A resposta atende exatamente ao que foi solicitado?
    2. **Coerência e Coesão:** O texto é logicamente estruturado e faz sentido?
    3. **Profundidade de Conteúdo:** A resposta demonstra compreensão profunda do tema?
    4. **Criatividade e Originalidade:** A resposta é inovadora e artística?
    5. **Fluência em Português:** A linguagem é natural, sem erros gramaticais ou ortográficos?

    Para cada critério, atribua uma nota de **1 a 5**, onde **1** é insuficiente e **5** é excelente. Forneça uma breve justificativa para cada nota.

    Por fim, apresente um **comentário geral** sobre a qualidade da resposta.
    """

    mensagem = f"""
    **Pergunta do Usuário:**

    {pergunta_usuario}

    **Resposta do Usuário:**

    {resposta_usuario}

    {criterios}
    """

    response = await client.chat.completions.create(
        model="gpt-4o-mini",  
        messages=[
            {"role": "system", "content": "Você é um avaliador especializado em análise de textos em língua portuguesa."},
            {"role": "user", "content": mensagem}
        ],
        temperature=0.2, 
        max_tokens=1024
    )

    return response.choices[0].message.content

In [24]:
async def process_responses(prompts, models, client_openai, responses):
    tasks = []

    max_concurrent_requests = 10 
    sem = asyncio.Semaphore(max_concurrent_requests)

    async def sem_avaliar_resposta(prompt, model, response):
        async with sem:
            max_retries = 3
            for attempt in range(max_retries):
                try:
                    eval_response = await avaliar_resposta(client_openai, prompt, response)
                    responses[prompt][model] = eval_response
                    print(f"\n\nAvaliação da resposta do modelo {model} para o prompt '{prompt}':\n\n{eval_response}\n\n")
                    break  # Se tudo der certo, saia do loop
                except client_openai.error.RateLimitError as e:
                    wait_time = 2 ** attempt  # Exponencial backoff
                    print(f"Limite de taxa atingido. Tentando novamente em {wait_time} segundos...")
                    await asyncio.sleep(wait_time)
                except Exception as e:
                    print(f"Erro ao avaliar a resposta para o modelo {model} e prompt '{prompt}': {e}")
                    break  # Se for outro erro, não tente novamente

    for prompt in prompts:
        for model in models:
            response = responses[prompt][model]
            task = asyncio.create_task(sem_avaliar_resposta(prompt, model, response))
            tasks.append(task)

    await asyncio.gather(*tasks)

In [25]:
# asyncio.run(process_responses(prompts, models, client_openai, responses))
await process_responses(prompts, models, client_openai, responses)





Avaliação da resposta do modelo llama-3.2-90b-text-preview para o prompt 'Escreva uma frase sobre a importância da leitura.':

**Avaliação da Resposta:**

1. **Relevância ao Prompt: 5/5**
   - A resposta atende exatamente ao que foi solicitado, apresentando uma frase que aborda diretamente a importância da leitura.

2. **Coerência e Coesão: 5/5**
   - A frase é bem estruturada e apresenta uma sequência lógica, facilitando a compreensão do leitor. Todos os elementos se conectam de forma fluida.

3. **Profundidade de Conteúdo: 3/5**
   - Embora a resposta mencione a importância da leitura, ela poderia explorar mais aspectos, como os benefícios cognitivos, emocionais e sociais que a leitura proporciona. Uma análise mais rica poderia enriquecer a compreensão do tema.

4. **Criatividade e Originalidade: 2/5**
   - A frase é bastante convencional e não apresenta uma abordagem inovadora. A falta de metáforas ou uma linguagem mais poética torna a resposta menos impactante e memorável.

5. **